In [1]:
import os
notebook_dir = os.getcwd() 
import sys
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..', 'src')))

import torch
from gnm import *
from gnm import defaults, utils, evaluation, fitting, generative_rules

# Performing sweeps

In this example notebook, we use the gnm package to perform a sweep over generative rules and parameter values.

## Loading in data

We'll start by loading in some data for our sweep. In particular, we'll need:
1. A distance matrix
2. A binary network to compare our networks to

We'll get these from the defaults sub-module, which has a built in distance matrix and consensus network.

In [2]:
distance_matrix = defaults.get_distance_matrix(device=torch.device('cpu'))
binary_consensus_network = defaults.get_binary_network(device=torch.device('cpu'))

We'll run the models until they have the same number of connections as the real binary consensus network

In [3]:
num_connections = int( binary_consensus_network.sum().item() / 2 )
print(f"The binary consensus network contains {num_connections} connections.")

The binary consensus network contains 400 connections.


## Defining our sweep

The next step is to define the parameters we want to sweep over. Here, we'll examine a range of values for $\eta$ and $\gamma$, for the two homophily-based wiring rules, the Matching Index rule. 

For each set of parameters, we'll generate 100 networks using the model with that set of parameters. This means setting the number of simulations to 100. 

In [4]:
binary_sweep_parameters = fitting.BinarySweepParameters(
    eta = torch.linspace(-1, 1, 6),
    gamma = torch.linspace(-1, 1, 6),
    lambdah = torch.Tensor([0.0]),
    distance_relationship_type = ["powerlaw"],
    preferential_relationship_type = ["powerlaw"],
    heterochronicity_relationship_type = ["powerlaw"],
    generative_rule = [generative_rules.MatchingIndex()],
    num_iterations = [num_connections],
)

num_simulations = 100

sweep_config = fitting.SweepConfig(
    binary_sweep_parameters = binary_sweep_parameters,
    num_simulations = num_simulations,
    distance_matrices = [distance_matrix],
)


## Creating our evaluations

We want to evaluate how good the fit of our models is the real binary consensus network. We'll use the standard evaluation criterion, which is the maximum of the KS statistics across the betweenness centrality, clustering coefficient, degree, and edge length distributions. 

In [5]:
criteria = [ evaluation.BetweennessKS(), evaluation.ClusteringKS(), evaluation.DegreeKS(), evaluation.EdgeLengthKS(distance_matrix) ]
energy = evaluation.MaxCriteria( criteria )

## Performing the sweep

In [ ]:
results = fitting.perform_sweep(sweep_config=sweep_config, 
                                binary_evaluations=[energy], 
                                real_binary_matrices=binary_consensus_network,
)

## Analysing the results

We can now analyse the sweep. 
The sweep outputs a list of Experiment dataclasses, which have two fields: run_config and results. 
We'll go through the sweep and find the energy for each parameter combination. 

In [ ]:
current_best_config = results[0].run_config
current_best_energy = results[0].results.binary_evaluations['Maximum(Binary betweenness centrality KS, Binary clustering coefficient KS, Binary degree KS, Binary edge length KS)'].mean()

for experiment in results:
    energies = experiment.results.binary_evaluations['Maximum(Binary betweenness centrality KS, Binary clustering coefficient KS, Binary degree KS, Binary edge length KS)']
    if energies.mean() < current_best_energy:
        current_best_config = experiment.run_config
        current_best_energy = energies.mean() 

plt.imshow()